In [ ]:
import sys
sys.path.append("../")

In [ ]:
import numpy as np
import pandas as pd
import qat.lang.AQASM as qlm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
from QQuantLib.utils.data_extracting  import get_results

## 1. CumulativeSumVar class

In order to simplify the conversion from the cumulative sum for var to a amplitude estimation we have developed the **CumulativeSumVaR** which is stored in the **var_benchmark.py** script.

For create the class we have to pass 2 arguments:

1. index: this is the index until we need to compute the cumulative sum 
2. An input dictionary where we can configure the complete VaR problem.

We can split the configuration info of the input dictionary in 4 main parts:

1. *Domain configuration*: this are the keys related where the domain of the problem (the x). The keys will be:
    * x0: initial value of the domain
    * xf: final value of the domain
    * n_qbits: for setting the number of parts for splitting the domain interval: $2^{n\_qbits}$
2. *Probability Configuration*: this are the keys for configurate the **DensityProbability** class of the *probability_class.py*. 
    * probability_type: string with the id of the porbability type. At this moment only the **Black-Scholes** is available. Users can add more distributions in the **DensityProbability** class.
    * s_0: parameter for the **Black-Scholes** density distribution (price of the active at time 0)
    * risk_free_rate: parameter for the **Black-Scholes** density distribution (risk free rate)
    * maturity: parameter for the **Black-Scholes** density distribution (time for evaluating the density distribution)
    * volatility: parameter for the **Black-Scholes** density distribution (volatility of the active)
3. *Data loadding Procedure*: This is only one key: **probability_loading** if True the **load_probability** function will be use for loading the probability and is False the **load_array** function will be used.
4. *Amplitude Estimation Configuration*: the main key is **ae_type** one where we select the AE method (can be: MLAE, CQPEAE, IQPEAE and RQAE). Additionally all the keys mandatory for configuring the method should be provided.

Following cell shows a typical dictionary for a **CumulativeSumVaR**.


In [ ]:
#Dictionary for CumulativeSumVaR class. In this case we are going to load the probability using the load_probability function
# and we are going to use the the MLAE class for solving the AE problem

m_k = [1, 100, 150, 200, 250, 300, 310, 320]

var_dict = {
    #Domain Configuration
    'x0': 0.01,
    'xf': 2.0,
    'n_qbits': 7,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'MLAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],  
    'delta' : 1.0e-6,
    'ns' : 10000,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': None    

}

In [ ]:
from var_benchmark import CumulativeSumVaR

Now we can call the class given the dictionary and an index until the cumulative sum want to be computed. 

**BE AWARE** The index **MUST BE** lower than the maximum lengh of the domain. If not a exception will be raised

In [ ]:
cumVar = CumulativeSumVaR(128, **var_dict)

When the class is called the following properties will be available:

* domain: array with the complete domain.
* probability: array with the probability properly configured following input keys
* index: input index

In [ ]:
cumVar = CumulativeSumVaR(60, **var_dict)

In [ ]:
plt.plot(cumVar.domain, cumVar.probability, '-')
plt.axvline(cumVar.domain[cumVar.index],)

## 2. The create oracle method.

Once the class is called we can invoque the **create_oracle** method that will create the oracle with the problem. Following properties are populated:

* var_oracle: QLM routine where the probability and the step function was loaded (this will be the oracle of the **AE** problem).
* p_gate: QLM routine for porbability loading
* step_gate: QLM routine for step function loading

In [ ]:
cumVar.create_oracle()

In [ ]:
print("Abstract Gate for Probability loading")
c = cumVar.p_gate
%qatdisplay c
print("Abstract Gate for step loading")
c = cumVar.step_gate
%qatdisplay c
print("Oracle QLM routine")
c = cumVar.var_oracle
%qatdisplay c

In [ ]:
results_loading, _, _, _ = get_results(cumVar.var_oracle, linalg_qpu=linalg_qpu, qubits=cumVar.co_index)

In [ ]:
results_loading

In [ ]:
np.sum(cumVar.probability[:cumVar.index])

### Loading Probability with load_array

In [ ]:
#Dictionary for CumulativeSumVaR class. In this case we are going to load the probability using the load_array function
# and we are going to use the the MLAE class for solving the AE problem

m_k = [1, 100, 150, 200, 250, 300, 310, 320]

var_dict_2 = {
    #Domain Configuration
    'x0': 0.01,
    'xf': 2.0,
    'n_qbits': 7,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'MLAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],  
    'delta' : 1.0e-6,
    'ns' : 10000,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': None    

}

In [ ]:
cumVar_2 = CumulativeSumVaR(60, **var_dict_2)

In [ ]:
plt.plot(cumVar_2.domain, cumVar_2.probability, '-')
plt.axvline(cumVar_2.domain[cumVar_2.index],)

In [ ]:
cumVar_2.create_oracle()

In [ ]:
print("Abstract Gate for Probability loading")
c = cumVar_2.p_gate
%qatdisplay c
print("Abstract Gate for step loading")
c = cumVar_2.step_gate
%qatdisplay c
print("Oracle QLM routine")
c = cumVar_2.var_oracle
%qatdisplay c

In [ ]:
results_loading, _, _, _ = get_results(cumVar_2.var_oracle, linalg_qpu=linalg_qpu)

In [ ]:
results_loading

In [ ]:
#Classical Result
np.sum(cumVar_2.probability[:cumVar_2.index])

In [ ]:
#Quantum Result
np.sqrt(results_loading.iloc[0]['Probability'])*2**cumVar_2.n_qbits

## 3. The run method.

The run method allows to compute (and return) the cumulative sum problem. The method can use an **AE** algorithm or the classical way dependign on the **ae_type** key. If this key is *MLAE, IQAE, RQAE, CQPEAE or IQPEAE* the correspondient algorithm will be used. Otherwise the classical computation will be used.

When a **AE** method is selecte a lot of properties are populated. Most important are:

* ae_pdf: This is the pure amplitude estimation results. It is a pandas dataframe with 3 columns:
    * ae : value of the pure **AE** estimation results
    * ae_l: lower limit for pure **AE** estimation results (only RQAE and IQAE)
    * ae_up: upper limit for pure **AE** estimation results (only RQAE and IQAE)

* ae_var: This is the resuls for the cumulative sum of the VaR (this is taking into account the way of the loading the probability and the **AE** method. Is a pandas dataframe with 3 columns:
    * Var_ae: value of cumulative sum for the Var
    * Var_ae_l: lower limit for the value of cumulative sum for the Var (only RQAE and IQAE)
    * Var_ae_u: upper limit for the value of cumulative sum for the Var (only RQAE and IQAE)
* classical_var : classical calculation of the cumulative sum    
* pdf: pandas DataFrame with the complete information of the VaR calculations. Most intersting columns are:
    * error_classical: difference between *Var_ae* and *classical_var*
    * relative_error_classical: ratio betwenn *error_classical* and *classical_var*
    * circuit_stasts: dictioanry with different statistics of the QLM ciruci
    * run_time: simulation time for running the **AE** algorithm

In [ ]:
from var_benchmark import CumulativeSumVaR

### 3.1 Classical Computation

For computing the cumulative sum using classical calculation only the info for the domain and the probability are mandatory. The **ae_type** is mandatory but CAN NOT BE *MLAE, IQAE, RQAE, CQPEAE or IQPEAE*. 

Following cell shows the input dictionary for classical computation.

In [ ]:
classical_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': None,    
    
}

In [ ]:
class_var = CumulativeSumVaR(15, **classical_dict)

In [ ]:
cum_sum = class_var.run()

print("Classical Cumulative Sum is: ", cum_sum)

### 3.2 AE methods: Loading Probability with load_probability

Now we can show how to use the class for different **AE** methods when the probability is loaded with the *load_probability* function (key **probability_loading** MUST BE True)

### MLAE

In [ ]:
m_k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

mlae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'MLAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],  
    'delta' : 1.0e-6,
    'ns' : 10000,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': None,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
mlae_var = CumulativeSumVaR(15, **mlae_dict)

In [ ]:
plt.plot(mlae_var.domain, mlae_var.probability, '-')
plt.axvline(mlae_var.domain[mlae_var.index],)

In [ ]:
cum_sum = mlae_var.run()
print("MLAE Cumulative Sum is: ", cum_sum)

In [ ]:
#Pure AE result
mlae_var.ae_pdf

In [ ]:
#VaR computed using AE
mlae_var.ae_var

In [ ]:
#Classical VaR computation
mlae_var.classical_var

In [ ]:
#Pandas DataFrame with the resuls
mlae_var.pdf

In [ ]:
mlae_var.pdf[
    [
        'ae_type', 'ae', 'Var_ae', 'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

### IQAE

In [ ]:
iqae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'IQAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': 0.05, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': 0.001,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
iqae_var = CumulativeSumVaR(15, **iqae_dict)
cum_sum = iqae_var.run()
print("IQAE Cumulative Sum is: ", cum_sum)

In [ ]:
iqae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

### CQPEAE

In [ ]:
cqpeae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'CQPEAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': 8,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
cqpeae_var = CumulativeSumVaR(15, **cqpeae_dict)
cum_sum = cqpeae_var.run()
print("CQPEAE Cumulative Sum is: ", cum_sum)

In [ ]:
cqpeae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

### IQPEAE

In [ ]:
iqpeae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'IQPEAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': 8,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': 20,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
iqpeae_var = CumulativeSumVaR(15, **iqpeae_dict)
cum_sum = iqpeae_var.run()
print("IQPEAE Cumulative Sum is: ", cum_sum)

In [ ]:
iqpeae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

### 3.2 Loading Probability with load_array

Now we can show how to use the class for different **AE** methods when the probability is loaded with the *load_probability* function (key **probability_loading** MUST BE False)

#### MLAE

In [ ]:
m_k = [1, 10, 50, 70, 90, 100, 110, 120]

mlae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'MLAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],  
    'delta' : 1.0e-6,
    'ns' : 10000,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': None,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
mlae_var = CumulativeSumVaR(15, **mlae_dict)
cum_sum = mlae_var.run()
print("MLAE Cumulative Sum is: ", cum_sum)

In [ ]:
mlae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

### IQAE

In [ ]:
iqae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'IQAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': 0.05, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': 0.0001,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
iqae_var = CumulativeSumVaR(15, **iqae_dict)
cum_sum = iqae_var.run()
print("IQAE Cumulative Sum is: ", cum_sum)

In [ ]:
iqae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical', 'run_time'
    ]
    
]

### RQAE

In [ ]:
rqae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'RQAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': 0.05,
    'q': 2.0,
    #IQAE and RQAE configuration    
    'epsilon': 0.0001,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
rqae_var = CumulativeSumVaR(15, **rqae_dict)
cum_sum = rqae_var.run()
print("RQAE Cumulative Sum is: ", cum_sum)

In [ ]:
rqae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical', 'run_time'
    ]
    
]

### CQPEAE

**BE AWARE**
For the probability loading using **load_array** the **CQPEAE** will need more *auxiliar_qbits_number* for getting a good precision. So the simulation in this case could be very long!!

In [ ]:
cqpeae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'CQPEAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': 8,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
cqpeae_var = CumulativeSumVaR(15, **cqpeae_dict)
cum_sum = cqpeae_var.run()
print("CQPEAE Cumulative Sum is: ", cum_sum)

In [ ]:
cqpeae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical', 'run_time'
    ]
    
]

### IQPEAE

**BE AWARE**
For the probability loading using **load_array** the **IQPEAE** will need more *cbits_number* for getting a good precision. So the simulation in this case could be very long!!

In [ ]:
iqpeae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'IQPEAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': 12,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': 20,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
iqpeae_var = CumulativeSumVaR(15, **iqpeae_dict)
cum_sum = iqpeae_var.run()
print("IQPEAE Cumulative Sum is: ", cum_sum)

In [ ]:
iqpeae_var.pdf[
    [
        'ae_type', 'ae', 'ae_l', 'ae_u',
        'Var_ae', 'Var_ae_l', 'Var_ae_u',
        'classical_var', 'error_classical', 'relative_error_classical'
    ]
    
]

## 4. The VaR class

Finally for doing the rest of the calculation mandatory for VaR the **VaR** class in the *var_benchmark* was created. 
The inputs for this class are:

1. alpha: is the $\alpha$ in the VaR problem.
2. input dictionary for configurate the **AE** problem for VaR. This dictionary is the same that the input dictionary for the **CumulativeSumVaR** class.

The run method of the **VaR** class executes the loop for computing the *VaR* as explained in the **03_Benchmark_VaR** notebook. For each step of the a **CumulativeSumVaR** class with the correspondient step index is created and its correspondient *run* method executed for getting the mandatory cumulative sum value. 

Several Properties are populated when the *run* method of the **VaR** class is executed:

* var_index: index of the probability arry that satisfy the VaR condition (the $\alpha$).
* var: value x of the domain that verifies: $P[X\leq x]\geq (1-\alpha)$.
* cumsum_var: is the cumulative sum of the probability until the VaR value: $P[X\leq x]$.
* list_cumsum: list with all the cumulative sums calculated during the VaR computation loop.
* cumsum_classes: list with all the **CumulativeSumVaR** class created during the VaR computation loop.


In [ ]:
from var_benchmark import VaR

### 4.1 Classic Var Computation

If *ae_type* kyey is different from *MLAE, IQAE, RQAE, CQPEAE or IQPEAE* the Var will be calculated using classical computation of cumulative sums

In [ ]:
classical_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': None,    
    
}

In [ ]:
classic_var = VaR(0.05, **classical_dict)

In [ ]:
classic_var.run()

In [ ]:
print('Index for Var: ',classic_var.var_index)
print('Var: ',classic_var.var)
print('Cum sum Var: ',classic_var.cumsum_var)
print('Test: ', classic_var.cumsum_var > 1.0- classic_var.alpha_var)

In [ ]:
#Cumulative intermediate sums
classic_var.list_cumsum

In [ ]:
#We can access to all the CumulativeSumVar class created durin VaR computation
cumsum_class = classic_var.cumsum_classes[0]

plt.plot(cumsum_class.domain, cumsum_class.probability)
plt.axvline(cumsum_class.domain[classic_var.var_index])

### 4.2 AE VaR computation

When the *ae_type* kyey is *MLAE, IQAE, RQAE, CQPEAE or IQPEAE* then the cumulative sums will be calculated using the correspondient AE method.

In [ ]:
iqae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'IQAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': 0.05, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': 0.001,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
iqae_var = VaR(0.05, **iqae_dict)
iqae_var.run()

In [ ]:
print('Index for Var: ',iqae_var.var_index)
print('Var: ',iqae_var.var)
print('Cum sum Var: ',iqae_var.cumsum_var)
print('Test: ', iqae_var.cumsum_var > 1.0- iqae_var.alpha_var)

In [ ]:
rqae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': False,
    
    #Amplitude Estimation selection
    'ae_type': 'RQAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        None,
        None
    ],  
    'delta' : None,
    'ns' : None,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': 0.05,
    'q': 2.0,
    #IQAE and RQAE configuration    
    'epsilon': 0.0001,
    'shots': 100,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
rqae_var = VaR(0.05, **rqae_dict)
rqae_var.run()

In [ ]:
print('Index for Var: ',rqae_var.var_index)
print('Var: ',rqae_var.var)
print('Cum sum Var: ',rqae_var.cumsum_var)
print('Test: ', rqae_var.cumsum_var > 1.0- rqae_var.alpha_var)

In [ ]:
m_k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

mlae_dict = {
    #Domain Configuration
    'x0': 0.1,
    'xf': 2.0,
    'n_qbits': 5,    
 
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.1,
    # Data loading Procedure
    'probability_loading': True,
    
    #Amplitude Estimation selection
    'ae_type': 'MLAE',    
    'mcz_qlm': False, #For using QLM multicontrolled Z
    #MLAE configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],  
    'delta' : 1.0e-6,
    'ns' : 10000,
    #CQPE_AE configuration
    'auxiliar_qbits_number': None,    
    #IQPE_AE configuration
    'cbits_number': None,
    #IQAE configuration
    'alpha': None, 
    #RQAE configuration    
    'gamma': None,
    'q': None,
    #IQAE and RQAE configuration    
    'epsilon': None,
    'shots': None,
    
    "qpu": linalg_qpu,
    "save": False
    

}

In [ ]:
mlae_var = VaR(0.05, **mlae_dict)
mlae_var.run()

In [ ]:
print('Index for Var: ',mlae_var.var_index)
print('Var: ',mlae_var.var)
print('Cum sum Var: ',mlae_var.cumsum_var)
print('Test: ', mlae_var.cumsum_var > 1.0- mlae_var.alpha_var)

## FINAL NOTES

Due to the high simulation times for **CQPEAE** and **IQPEAE** this methods are not recommende to use with a complete VaR computation!!!